In [1]:
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import NLTKWordTokenizer
import string

In [88]:
df =  pd.DataFrame(json.load(open("new_train.json", "r")), columns=['id', 'token', 'author'])
df.head()

,id,token,author
0,id10914,"It will have been, by that time, totally destr...",EAP
1,id03295,"Make me happy, and I shall again be virtuous.""",MWS
2,id04103,"The margin of the river, and of the many dazzl...",EAP
3,id26189,The heat became all at once intolerable.,EAP
4,id07955,"What, indeed, was my passion for the young gir...",EAP


In [89]:
df.token = df.token.apply(lambda x: x.lower())
df.token = df.token.apply(lambda s: [s[start:end] for start, end in NLTKWordTokenizer().span_tokenize(s)])
df.token = df.token.apply(lambda doc: [w for w in doc if not w in string.punctuation])
df.head()

,id,token,author
0,id10914,"[it, will, have, been, by, that, time, totally...",EAP
1,id03295,"[make, me, happy, and, i, shall, again, be, vi...",MWS
2,id04103,"[the, margin, of, the, river, and, of, the, ma...",EAP
3,id26189,"[the, heat, became, all, at, once, intolerable]",EAP
4,id07955,"[what, indeed, was, my, passion, for, the, you...",EAP


In [90]:
sentence_len = pd.DataFrame(df)
sentence_len.token = df.token.apply(lambda x: len(x))
sentence_len = sentence_len.groupby("author").agg(
    Satzlänge= pd.NamedAgg("token", aggfunc="mean")
).reset_index()
results = sentence_len.pivot_table(index = "author", values="Satzlänge").reset_index()

In [91]:

results["Anzahl Sätze"] = df.groupby("author").agg(
    count=pd.NamedAgg("token", aggfunc="count")
).pivot_table(index = "author", values="count").reset_index()["count"]
results

,author,Satzlänge,Anzahl Sätze
0,EAP,25.517697,6357
1,HPL,27.902673,4490
2,MWS,27.452035,4816


In [92]:

results["Anzahl Wörter"] = df.groupby("author").agg(
    count=pd.NamedAgg("token", aggfunc=lambda x: len(x.explode().dropna()))
).pivot_table(index = "author", values="count").reset_index()["count"]
results

,author,Satzlänge,Anzahl Sätze,Anzahl Wörter
0,EAP,25.517697,6357,162216
1,HPL,27.902673,4490,125283
2,MWS,27.452035,4816,132209


In [93]:
results["Vokabulargröße"] = df.groupby("author").agg(
    count=pd.NamedAgg("token", aggfunc=lambda x: len(x.explode().dropna().unique()))
).pivot_table(index = "author", values="count").reset_index()["count"]
results

,author,Satzlänge,Anzahl Sätze,Anzahl Wörter,Vokabulargröße
0,EAP,25.517697,6357,162216,14145
1,HPL,27.902673,4490,125283,13198
2,MWS,27.452035,4816,132209,10503


In [96]:
results.loc[3, "author"] = "Korpus"
results.loc[3, "Satzlänge"] = df.token.apply(lambda x: len(x)).mean()
results.loc[3, "Anzahl Sätze"] = len(df.token.dropna())
results.loc[3, "Anzahl Wörter"] = len(df.token.explode().dropna())
results.loc[3, "Vokabulargröße"] = len(df.token.explode().dropna().unique())
results = results.rename(columns={"author": "Datensatz"})
results["Datensatz"] = ["EEdgar Allan Poe", "H. P. Lovecraft", "Mary Shelley", "Korpus"]
display(results)
results.to_latex("korpus.tex")

,Datensatz,Satzlänge,Anzahl Sätze,Anzahl Wörter,Vokabulargröße,Datensatz,Datensatz
0,EEdgar Allan Poe,25.517697,6357.0,162216.0,14145.0,EEdgar Allan Poe,EEdgar Allan Poe
1,H. P. Lovecraft,27.902673,4490.0,125283.0,13198.0,H. P. Lovecraft,H. P. Lovecraft
2,Mary Shelley,27.452035,4816.0,132209.0,10503.0,Mary Shelley,Mary Shelley
3,Korpus,26.796144,15663.0,419708.0,23366.0,Korpus,Korpus


/tmp/ipykernel_5176/1918086178.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.to_latex("korpus.tex")
